<a href="https://colab.research.google.com/github/navneetkrc/Deep-Learning-Experiments-implemented-using-Google-Colab/blob/master/QA_Generation_using_GeminiAPI_from_Support_Docs1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## pymupdf4llm

In [1]:
!pip install pymupdf4llm==0.0.17 > /dev/null 2>&1

In [2]:
!wget https://downloadcenter.samsung.com/content/UM/202501/20250124164253078/SM-S93X_UG_EU_15_Eng_Rev.1.0_250123.pdf

--2025-01-28 18:22:32--  https://downloadcenter.samsung.com/content/UM/202501/20250124164253078/SM-S93X_UG_EU_15_Eng_Rev.1.0_250123.pdf
Resolving downloadcenter.samsung.com (downloadcenter.samsung.com)... 23.213.219.134
Connecting to downloadcenter.samsung.com (downloadcenter.samsung.com)|23.213.219.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3451988 (3.3M) [application/pdf]
Saving to: ‘SM-S93X_UG_EU_15_Eng_Rev.1.0_250123.pdf’

SM-S93X_UG_EU_15_En 100%[===================>]   3.29M  5.24MB/s    in 0.6s    

2025-01-28 18:22:33 (5.24 MB/s) - ‘SM-S93X_UG_EU_15_Eng_Rev.1.0_250123.pdf’ saved [3451988/3451988]



In [3]:
import pymupdf4llm

markdwon_text = pymupdf4llm.to_markdown('/content/SM-S93X_UG_EU_15_Eng_Rev.1.0_250123.pdf') #S25 User Manual

Processing /content/SM-S93X_UG_EU_15_Eng_Rev.1.0_250123.pdf...
[                                        ] (0/185)[                                        ] (  1/185)[                                        ] (  2/185)[                                        ] (  3/185)[                                        ] (  4/185)=[=                                       ] (  5/185)[=               

In [4]:
generated_text = markdwon_text
content = markdwon_text
generated_text

"# USER GUIDE\n\n###### SM-S931B SM-S931B/DS SM-S936B SM-S936B/DS SM-S938B SM-S938B/DS\n\n English 01/2025 Rev1 0 www samsung com\n\n\n-----\n\n## Table of Contents\n\n\n#### Getting started\n\n###### 5 Device layout and functions 11 Charging the battery 16 Nano-SIM card and eSIM 18 Turning the device on and off 19 Initial setup 20 Samsung account 21 Transferring data from your previous device (Smart Switch) 22 Understanding the screen 30 Notification panel 31 Quick settings panel 33 Screen capture and screen record 35 Entering text 37 Extracting text\n\n#### Apps and features\n\n###### 38 Installing or uninstalling apps 39 Galaxy AI 47 S Pen (Galaxy S25 Ultra) 56 Phone 60 Contacts 63 Camera 81 Gallery 86 Bixby 87 Bixby Vision 88 Multi window (Using multiple apps at once)\n\n\n###### 89 Samsung Internet 90 Samsung Wallet 94 Samsung Health 94 Samsung Notes 99 Samsung Members 99 Samsung Kids 100 Samsung Global Goals 100 Samsung TV Plus 100 Samsung Find 100 Samsung Shop 101 Galaxy Wearabl

In [5]:
def test_qa_generators(base_function_name, content, suffixes=["", "_deepseek", "_google", "_sonnet"]):
    """Tests and prints output for different QA generation functions."""
    for suffix in suffixes:
        function_name = base_function_name + suffix
        try:
            qa_function = globals()[function_name] # Get function from global scope
            print(f"\n--- Calling {function_name} ---")
            qa_output = qa_function(content)
            print(qa_output)
        except KeyError:
            print(f"\n--- Function {function_name} not found ---")
        except Exception as e:
            print(f"\n--- Error in {function_name} ---")
            print(f"Error details: {e}")



In [6]:
def test_qa_generators_aggregated(base_function_name, content, suffixes=["", "_deepseek", "_google", "_sonnet"]):
    """
    Tests different QA generation functions and returns aggregated results in a single JSON structure.

    Args:
        base_function_name (str): Base name of the QA generation function
        content (str): Content to process
        suffixes (list): List of suffixes for different model variants

    Returns:
        dict: Aggregated results with the following structure:
        {
            "base_function": str,
            "results": {
                "model_variant": result_dict,
                ...
            },
            "errors": {
                "model_variant": error_message,
                ...
            }
        }
    """
    aggregated_results = {
        "base_function": base_function_name,
        "results": {},
        "errors": {}
    }

    for suffix in suffixes:
        function_name = base_function_name + suffix
        model_variant = suffix if suffix else "default"

        try:
            qa_function = globals()[function_name]  # Get function from global scope
            qa_output = qa_function(content)
            aggregated_results["results"][model_variant] = qa_output

        except KeyError:
            error_msg = f"Function {function_name} not found"
            aggregated_results["errors"][model_variant] = error_msg

        except Exception as e:
            error_msg = f"Error in execution: {str(e)}"
            aggregated_results["errors"][model_variant] = error_msg

    return aggregated_results

# Example usage and pretty printing
import json

def print_aggregated_results(results):
    """Helper function to pretty print the aggregated results."""
    print(json.dumps(results, indent=2))

## QA Pair Generation using Gemini *API*



In [7]:
import google.generativeai as genai
from google.colab import userdata
apikey=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=apikey)

### Number 1 generate_structured_qa_pairs

In [8]:
#original
def generate_simple_qa_pairs(text):
    """General informative Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate fundamental Q&A pairs from Samsung support docs covering:
    - Key features - Basic troubleshooting - Usage guidelines
    Use consumer-friendly language with concise answers (1-2 sentences).
    Text: {text}
    """)
    return model.generate_content(prompt).text


#Deepseek R1
import json
from textwrap import dedent
import google.generativeai as genai

def generate_simple_qa_pairs_deepseek(text):
    """Generates Q&A pairs in a standardized JSON format.

    Returns:
        A JSON object with nested "Generations", "Questions", and "Answers".
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate Q&A pairs from the Samsung support document below.
    Follow these rules:
    1. Include key features, troubleshooting steps, and usage guidelines.
    2. Use short, conversational questions and answers (1-2 sentences).
    3. Format output as JSON with a "Generations" key containing a list of Q&A objects.
    4. Structure each object as {{"Question": "...", "Answer": "..."}}.

    Example:
    {{
      "Generations": [
        {{"Question": "How do I restart my Galaxy phone?", "Answer": "Hold the power and volume down buttons for 10 seconds."}},
        {{"Question": "How to extend battery life?", "Answer": "Enable Power Saving Mode in Settings > Battery."}}
      ]
    }}

    Text: {text}
    """)

    response = model.generate_content(prompt).text
    # Remove markdown formatting (common in Gemini outputs)
    response = response.strip("` \n").replace("json", "")
    return json.loads(response)


#Google AI STudio
import google.generativeai as genai
from textwrap import dedent
import json

def generate_simple_qa_pairs_google(text):
    """
    Generates informative Q&A pairs from Samsung support docs and returns them in JSON format.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate fundamental Question and Answer pairs from the following Samsung support document text.
    Focus on:
    - Key features
    - Basic troubleshooting steps
    - Usage guidelines

    Format each question and answer pair clearly. Aim for consumer-friendly language and keep answers concise (1-2 sentences).

    Present the output as a list of dictionaries. Each dictionary should have the keys "Question" and "Answer".

    Text: {text}

    Output Format:
    [
      {{"Question": "...", "Answer": "..."}},
      {{"Question": "...", "Answer": "..."}},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        qa_list = json.loads(response_text) # Attempt to parse the response as JSON
    except json.JSONDecodeError:
        # If direct JSON parsing fails, fallback to a more robust extraction method
        print("Warning: Failed to parse JSON directly. Attempting to extract Q&A pairs from text.")
        # This fallback is a basic example, you might need more sophisticated parsing
        qa_list = []
        lines = response_text.strip().split('\n')
        for i in range(0, len(lines), 2): # Assuming Q and A alternate lines
            if i + 1 < len(lines):
                question = lines[i].strip().replace("Question:", "").strip() if "Question:" in lines[i] else lines[i].strip()
                answer = lines[i+1].strip().replace("Answer:", "").strip() if "Answer:" in lines[i+1] else lines[i+1].strip()
                qa_list.append({"Question": question, "Answer": answer})


    # Structure the output in the desired JSON format
    output_json = {
        "Generations": [
            {
                "Questions": [qa_pair["Question"] for qa_pair in qa_list],
                "Answers": [qa_pair["Answer"] for qa_pair in qa_list]
            }
        ]
    }
    return json.dumps(output_json, indent=2) # Return JSON string with indent for readability


#Sonnet
def generate_simple_qa_pairs_sonnet(text):
    """Generate structured Q&A pairs in JSON format"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate comprehensive Q&A pairs from Samsung support docs in strict JSON format:
    {{
      "Generations": {{
        "Questions": [
          "question1",
          "question2"
        ],
        "Answers": [
          "answer1",
          "answer2"
        ],
        "Metadata": {{
          "source": "Samsung support document",
          "coverage": ["features", "troubleshooting", "usage"]
        }}
      }}
    }}

    Criteria:
    - Generate 3-5 questions
    - Answers in 1-2 concise sentences
    - Use consumer-friendly language
    - Ensure direct mapping between questions and answers

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        # Fallback parsing if direct JSON parsing fails
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }


In [ ]:
# Example Usage:
print("--- Simple Question-Answer Pairs ---")
test_qa_generators("generate_simple_qa_pairs", content)

--- Simple Question-Answer Pairs ---

--- Calling generate_simple_qa_pairs ---
## Samsung Galaxy S25 Series Q&A:

**Key Features:**

**Q: What is the S Pen used for (Galaxy S25 Ultra)?**
A: The S Pen lets you write, draw, take notes, control the phone remotely, and access various features using air gestures.

**Q: What is Bixby?**
A: Bixby is Samsung's AI assistant; you can use voice commands to control your phone and access information.

**Q: What is Samsung DeX?**
A: Samsung DeX turns your phone into a desktop computer experience when connected to a monitor.

**Q: What is Samsung Wallet?**
A: Samsung Wallet lets you store credit cards, digital keys, boarding passes, and more.


**Basic Troubleshooting:**

**Q: My phone won't turn on.**
A: Fully charge the battery before trying again.  If it still doesn't work, try a forced restart.

**Q: My touchscreen is unresponsive.**
A: Restart your phone. If the issue persists, check for screen damage or interference from screen protectors.

**Q

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3545.10ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3756.50ms


In [ ]:
results = test_qa_generators_aggregated("generate_structured_qa_pairs", content)
print_aggregated_results(results)

### Number 2 generate_structured_qa

In [11]:
#original
def generate_structured_qa(text):
    """Problem-solution focused pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create problem-solution pairs from Samsung docs. Format:
    Question: [Issue description]
    Answer: [Step-by-step fix]
    Include severity indicators (⚠️) for critical issues.
    Text: {text}
    """)
    return model.generate_content(prompt).text


#Deepseek R1
import json
from textwrap import dedent

def generate_structured_qa_deepseek(text):
    """Generates severity-graded troubleshooting pairs in JSON format.

    Returns:
        JSON with error codes, time estimates, and step-by-step instructions.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Convert Samsung support text into problem-solution pairs with:
    1. Technical error codes (if applicable)
    2. Severity grading (Critical/Medium/Low)
    3. Estimated troubleshooting time
    4. Ordered resolution steps

    Format requirements:
    - JSON with "Generations" containing:
      {{
        "Question": "[Issue]",
        "Answer": {{
          "Steps": ["1. Do X", "2. Then Y"],
          "SeverityCode": 0-2 (2=Critical),
          "ErrorCode": "ERR_BATT_01",
          "TroubleshootingTime": "5-7 minutes"
        }}
      }}
    - Replace ⚠️ with "SeverityCode":2
    - Sort by descending SeverityCode

    Example:
    {{
      "Generations": [
        {{
          "Question": "Phone overheating during charging",
          "Answer": {{
            "Steps": ["1. Unplug immediately", "2. Remove case", "3. Contact support"],
            "SeverityCode": 2,
            "ErrorCode": "ERR_CHG_OVERHEAT",
            "TroubleshootingTime": "3 minutes"
          }}
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse troubleshooting data"}


#Google AI STudio
def generate_structured_qa_google(text):
    """Problem-solution focused pairs in JSON format with severity indicators"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate problem-solution Question and Answer pairs from the following Samsung support document text. Format each pair as:

    Question: [Issue description]
    Answer: [Step-by-step fix]

    For critical issues, include a severity indicator (⚠️) in the output and classify them as "Critical" in the JSON structure. For non-critical issues, classify as "Normal" or leave severity blank/null.

    Use consumer-friendly language. Keep answers concise and step-by-step where possible.

    Present the output as a list of dictionaries in JSON format. Each dictionary should have keys: "Severity" (e.g., "Critical", "Normal", null), "Question", "Answer", and optionally "SeverityIndicator" (the ⚠️ symbol if applicable).

    Text: {text}

    Output Format (example):
    [
      {{ "Severity": "Critical", "Question": "...", "Answer": "...", "SeverityIndicator": "⚠️" }},
      {{ "Severity": "Normal", "Question": "...", "Answer": "...", "SeverityIndicator": null }},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        problem_solution_data = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        problem_solution_data = [] # Implement a robust fallback text parsing

    output_json = {
        "Generations": [
            {"ProblemSolutionPairs": problem_solution_data} # Assuming model outputs list of dictionaries as requested
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet
def generate_structured_qa_sonnet(text):
    """Generate problem-solution Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate problem-solution Q&A in JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Specific problem description"
        ],
        "Answers": [
          "Comprehensive step-by-step solution"
        ],
        "Metadata": {{
          "IssueSeverity": ["Low", "Medium", "High"],
          "SolutionComplexity": "Detailed technical guidance"
        }},
        "IssueClassification": {{
          "CriticalIssues": ["Urgent problems requiring immediate action"],
          "WarningIndicators": ["⚠️ Severity markers"]
        }}
      }}
    }}

    Requirements:
    - Precise problem descriptions
    - Clear, sequential solution steps
    - Include severity indicators
    - 1-2 sentence answers
    - Technical accuracy

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }


In [12]:
# Example Usage:
print("--- generate_structured_qa ---")
test_qa_generators("generate_structured_qa", content)

--- generate_structured_qa ---

--- Calling generate_structured_qa ---
## Samsung Galaxy S25 Problem-Solution Pairs

Here are some problem-solution pairs extracted from the provided Samsung document, categorized for clarity and including severity indicators where appropriate:

**I. Device Hardware & Physical Issues:**

**Question:** My device's sound is quiet or certain features aren't working.
**Answer:** Dust or foreign materials may have entered the microphone, speaker, or receiver.  Do *not* use a sharp object to remove them, as this could damage the device.  If the issue persists, contact a Samsung Service Centre.

**Question:** My camera images have dots or spots.
**Answer:** The camera lens may have been exposed to a strong light source (e.g., direct sunlight), damaging the image sensor. This damage is irreparable.

**Question:** I'm experiencing connectivity problems and battery drain.
**Answer:** This may be due to metallic stickers on the antenna area, a metallic device cover

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2635.47ms


{
  "Generations": [
    {
      "ProblemSolutionPairs": []
    }
  ]
}

--- Calling generate_structured_qa_sonnet ---
{'Generations': {'Questions': [], 'Answers': [], 'RawResponse': '```json\n{\n  "Generations": {\n    "Questions": [\n      "My Samsung Galaxy S25 Ultra\'s camera lens is exposed to direct sunlight. What should I do?",\n      "My Samsung Galaxy S25+ is not charging wirelessly. What are the possible causes and solutions?",\n      "I forgot my Samsung account password. How can I recover it?",\n      "How do I take a screenshot on my Samsung Galaxy S25?",\n      "My Samsung Galaxy S25 is overheating while playing a game. What steps can I take to resolve this?",\n      "How can I use Bixby Vision to translate text from an image?",\n      "I want to use two WhatsApp accounts on my phone. How can I do this using Dual Messenger?",\n      "My phone\'s battery is draining too quickly. What can I do to improve battery life?",\n      "How do I share a contact from my Samsung Galax

### Number 3 generate_step_by_step_qa

In [13]:
#original
def generate_step_by_step_qa(text):
    """Procedural guidance pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Extract step-by-step procedures and generate Q&A pairs with:
    - Sequential questions - Tool requirements - Time estimates
    Format steps with numbered sub-items.
    Text: {text}
    """)
    return model.generate_content(prompt).text


#Deepseek R1
import json
from textwrap import dedent

def generate_step_by_step_qa_deepseek(text):
    """Generates machine-parseable procedural guides with safety checks.

    Returns:
        JSON with nested step details, tool SKUs, and critical warnings.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Convert Samsung technical docs into structured Q&A with:
    1. Sequentially ordered repair/configuration steps
    2. Official tool SKUs from Samsung Parts
    3. Time estimates with ranges (e.g., "8-12 minutes")
    4. Critical warnings for permanent data loss

    Format requirements:
    - JSON with "Generations" containing:
      {{
        "Question": "[Task description]",
        "Answer": {{
          "Steps": [
            {{
              "Number": 1,
              "Action": "...",
              "Tools": ["GJ-TX42 (SIM ejector)"],
              "Warning": "Optional text"
            }}
          ],
          "TotalTime": "15-20 mins",
          "RiskLevel": "High/Medium/Low"
        }},
        "Category": "Display/Battery/Software"
      }}
    - Sort steps by dependency (prerequisite first)
    - Validate tool SKUs against Samsung's GJ-XXXXX format

    Example:
    {{
      "Generations": [
        {{
          "Question": "How to replace Galaxy S24 Ultra screen?",
          "Answer": {{
            "Steps": [
              {{
                "Number": 1,
                "Action": "Heat edges at 85°C with GJ-HX90 thermal pad",
                "Tools": ["GJ-HX90"],
                "Warning": "Exceeding 90°C voids warranty"
              }}
            ],
            "TotalTime": "45-60 mins",
            "RiskLevel": "High"
          }},
          "Category": "Display"
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse procedural guide"}


#Google AI STudio
def generate_step_by_step_qa_google(text):
    """Procedural guidance pairs in JSON format with steps, tools, and time estimates"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Extract step-by-step procedures from the following Samsung support document text and generate Question and Answer pairs for each step.

    For each procedure, identify:
    - Sequential questions that guide the user through each step.
    - Any tool requirements needed for the procedure.
    - An estimated time to complete the procedure.

    Format the steps as numbered sub-items in the plain text output for clarity. In the JSON output, represent steps as a numbered list of Question-Answer pairs.

    Use consumer-friendly language. Keep answers concise and focused on each step.

    Present the output as a list of procedures in JSON format. Each procedure should include "ProcedureName", "ToolRequirements", "TimeEstimate", and a list of "Steps", where each step has "StepNumber", "Question", and "Answer".

    Text: {text}

    Output Format (example structure):
    [
      {{
        "ProcedureName": "Procedure Title",
        "ToolRequirements": ["Tool 1", "Tool 2", ...],
        "TimeEstimate": "Estimated Time",
        "Steps": [
          {{ "StepNumber": 1, "Question": "Question for Step 1?", "Answer": "Answer for Step 1" }},
          {{ "StepNumber": 2, "Question": "Question for Step 2?", "Answer": "Answer for Step 2" }},
          ...
        ]
      }},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        procedure_data = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        procedure_data = [] # Implement a robust fallback text parsing

    output_json = {
        "Generations": [
            {"ProcedureQAPairs": procedure_data} # Assuming model outputs list of procedures as requested
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet
def generate_step_by_step_qa_sonnet(text):
    """Generate structured procedural guidance Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create step-by-step Q&A in JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Procedural guidance question"
        ],
        "Answers": [
          "Detailed step-by-step procedure"
        ],
        "Metadata": {{
          "ProcedureType": "Sequential Guidance",
          "Complexity": "Detailed Technical Steps"
        }},
        "ProcedureDetails": {{
          "Tools": ["Required equipment"],
          "TimeEstimates": ["Estimated duration for each step"],
          "Steps": [
            "Numbered sequential instructions"
          ]
        }}
      }}
    }}

    Requirements:
    - Numbered, sequential steps
    - Tool and time requirement details
    - 1-2 sentence answers
    - Technical precision

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }

In [14]:
# Example Usage:
print("--- generate_step_by_step_qa ---")
test_qa_generators("generate_step_by_step_qa", content)

--- generate_step_by_step_qa ---

--- Calling generate_step_by_step_qa ---
## Samsung Galaxy S25 User Guide: Step-by-Step Procedures and Q&A

This document extracts step-by-step procedures and generates Q&A pairs from the provided Samsung Galaxy S25 User Guide.


**I. Getting Started**

**A. Device Layout and Functions (Page 11)**

**Step-by-Step:**  This section provides diagrams, not numbered steps for device layout.  The text describes the function of buttons.

1. **Side Button:**  Press and hold to turn on (when off). Press to turn on/lock screen. Double press or press and hold to open a custom app/feature (set in settings).
2. **Side Button + Volume Down Button:** Press simultaneously to capture a screenshot.
3. **Volume Down Button + Side Button:** Press and hold simultaneously to turn off the device.

**Q&A:**

Q1: How do I turn on my Galaxy S25?
A1: Press and hold the Side button for a few seconds.

Q2: How do I take a screenshot?
A2: Press the Side button and Volume Down butto

### Number 4 generate_faq_qa

In [15]:
#original
def generate_faq_qa(text):
    """Anticipated user questions"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Predict common FAQs from Samsung users based on this text.
    Phrase questions as natural customer inquiries.
    Include 'Why', 'How', and 'What if' scenarios.
    Text: {text}
    """)
    return model.generate_content(prompt).text


#Deepseek R1
import json
from textwrap import dedent

def generate_faq_qa_deepseek(text):
    """Generates categorized FAQs with popularity metadata in JSON format.

    Returns:
        JSON with question taxonomy, search keywords, and related products.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate structured FAQs from Samsung documentation with:
    1. Natural phrasing mirroring customer voice
    2. Taxonomy:
       - "Type": ["Why", "How", "What If", "Setup", "Troubleshooting"]
       - "FrequencyScore": 1-5 (5=most common)
    3. Related product models
    4. Search keywords (3-5 per question)

    Format requirements:
    - JSON with "Generations" containing:
      {{
        "Question": "...",
        "Answer": "...",
        "Metadata": {{
          "Type": "Why",
          "FrequencyScore": 4,
          "RelatedProducts": ["SM-S901B", "QE65QN900B"],
          "Keywords": ["battery drain", "optimization"]
        }}
      }}
    - Sort by descending FrequencyScore
    - Deduplicate similar questions

    Example:
    {{
      "Generations": [
        {{
          "Question": "Why does my Galaxy phone get hot during video calls?",
          "Answer": "Processor-intensive apps...",
          "Metadata": {{
            "Type": "Why",
            "FrequencyScore": 5,
            "RelatedProducts": ["SM-S918B", "SM-S711B"],
            "Keywords": ["overheating", "thermal", "video chat"]
          }}
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse FAQ data"}


#Google AI STudio
def generate_faq_qa_google(text):
    """Generates anticipated user questions (FAQs) and answers in JSON format."""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Predict common Frequently Asked Questions (FAQs) from Samsung users based on the following support document text.

    Phrase the questions as natural customer inquiries, reflecting how a user might actually ask.
    Include 'Why', 'How', and 'What if' scenarios to cover a range of common question types.
    For each question, also generate a concise and helpful answer.
    Use consumer-friendly language for both questions and answers.

    Present the output as a list of FAQ Question-Answer pairs in JSON format. Each pair should have the keys "Question" and "Answer".

    Text: {text}

    Output Format (example):
    [
      {{ "Question": "...", "Answer": "..." }},
      {{ "Question": "...", "Answer": "..." }},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        faq_data = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        faq_data = [] # Implement a robust fallback text parser

    output_json = {
        "Generations": [
            {"FAQs": faq_data} # Assuming model outputs a list of dictionaries as requested
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet

def generate_faq_qa_sonnet(text):
    """Generate structured user FAQ Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create FAQ Q&A in JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Natural customer inquiry using 'Why', 'How', 'What if'"
        ],
        "Answers": [
          "Clear, concise response addressing user concern"
        ],
        "Metadata": {{
          "QuestionTypes": ["Why", "How", "What if"],
          "UserPerspective": "Common customer inquiries"
        }},
        "FAQCategories": [
          "Common user pain points",
          "Anticipated information needs"
        ]
      }}
    }}

    Requirements:
    - Realistic, conversational questions
    - Comprehensive yet brief answers
    - Cover multiple inquiry types
    - 1-2 sentence responses

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }

In [16]:
# Example Usage:
print("--- generate_faq_qa ---")
test_qa_generators("generate_faq_qa", content)

--- generate_faq_qa ---

--- Calling generate_faq_qa ---


ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2780.29ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2579.00ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2106.45ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2250.68ms


Here are some common FAQs from Samsung users, based on the provided user guide text, categorized for clarity:

**I. Getting Started & Initial Setup:**

* **Why won't my phone turn on?**  (Possible answers: completely discharged battery, software issue, hardware malfunction)
* **How do I transfer my data from my old phone to my new Samsung phone?** (Answer: Use Smart Switch, either via USB cable or wirelessly.)
* **What if I forget my Samsung account password or ID?** (Answer: Use the "Forgot password" or "Forgot ID" options during sign-in.)
* **How do I set up my new Samsung phone for the first time?** (Answer: Follow the on-screen instructions after turning it on.)
* **Why is my phone asking for a PIN?** (Possible answers: SIM card requires a PIN, security setting enabled)
* **How do I insert my nano-SIM card?** (Answer: Refer to the guide's diagram and instructions for SIM tray insertion.)
* **What if I damage the SIM card tray while inserting or removing the SIM card?** (Answer:  Se

### Number 5 generate_troubleshooting_qa

In [17]:
#original
def generate_troubleshooting_qa(text):
    """Diagnostic-focused pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create advanced troubleshooting pairs including:
    - Error symptoms - Diagnostic checks - Repair procedures
    Format solutions with priority levels (Critical/High/Medium).
    Text: {text}
    """)
    return model.generate_content(prompt).text


#Deepseek R1
import json
from textwrap import dedent

def generate_troubleshooting_qa_deepseek(text):
    """Generates machine-actionable diagnostic guides with error codes.

    Returns:
        JSON with nested symptom analysis, error codes, and repair workflows.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create advanced troubleshooting pairs from Samsung docs with:
    1. Symptom pattern matching (e.g., "Boot loop after OS update")
    2. Diagnostic codes following TROUB-XXXX format
    3. Repair steps with required tools/time
    4. Priority matrix (Critical=hardware safety risk)

    Format requirements:
    - JSON with "Generations" containing:
      {{
        "Question": "[Observed symptom]",
        "Answer": {{
          "Symptoms": ["Symptom 1", "Symptom 2"],
          "Diagnostics": ["Check X", "Test Y"],
          "RepairSteps": ["1. Replace component", "2. Flash firmware"],
          "Priority": "Critical",
          "ErrorCode": "TROUB-0238",
          "ToolsRequired": ["GJ-FT55", "Multimeter"],
          "AvgRepairTime": "45 mins"
        }},
        "RelatedComponents": ["Battery", "Motherboard"]
      }}
    - Sort by Priority then AvgRepairTime
    - Validate ErrorCode format: TROUB- followed by 4 digits

    Example:
    {{
      "Generations": [
        {{
          "Question": "Phone restarts randomly during calls",
          "Answer": {{
            "Symptoms": ["Thermal throttling", "Signal drops"],
            "Diagnostics": ["Check battery swelling", "Run ANT+ test"],
            "RepairSteps": ["Replace battery module SPX-897"],
            "Priority": "Critical",
            "ErrorCode": "TROUB-1151",
            "ToolsRequired": ["GJ-TH92 thermal gun"],
            "AvgRepairTime": "25 mins"
          }},
          "RelatedComponents": ["Battery", "Antenna"]
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse diagnostic protocol"}


#Google AI STudio
def generate_troubleshooting_qa_google(text):
    """Advanced troubleshooting pairs in JSON format with priority levels."""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate advanced troubleshooting Question and Answer pairs from the following Samsung support document text.

    Include for each troubleshooting pair:
    - Error symptoms (brief description)
    - Diagnostic checks to perform
    - Repair procedures or resolution steps

    Format each solution with a priority level: "Critical", "High", or "Medium" based on the severity and urgency of the issue.

    Use consumer-friendly language. Keep answers concise and step-by-step where applicable.

    Present the output as a list of troubleshooting Q&A pairs in JSON format. Each pair should be a dictionary with keys: "Priority" ("Critical", "High", "Medium"), "Question" (error symptom description), "Answer" (troubleshooting/resolution steps), and optionally "ErrorSymptoms", "DiagnosticChecks", "RepairProcedures" (as lists of strings).

    Text: {text}

    Output Format (example):
    [
      {{ "Priority": "Critical", "Question": "...", "Answer": "...", "ErrorSymptoms": [...], "DiagnosticChecks": [...], "RepairProcedures": [...] }},
      {{ "Priority": "Medium", "Question": "...", "Answer": "...", "ErrorSymptoms": [...], "DiagnosticChecks": [...], "RepairProcedures": [...] }},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        troubleshooting_data = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        troubleshooting_data = [] # Implement robust fallback parsing

    output_json = {
        "Generations": [
            {"TroubleshootingQAPairs": troubleshooting_data} # Assuming model outputs list of dictionaries
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet
def generate_troubleshooting_qa_sonnet(text):
    """Generate structured diagnostic Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create troubleshooting Q&A in JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Specific error or diagnostic scenario"
        ],
        "Answers": [
          "Comprehensive troubleshooting solution"
        ],
        "Metadata": {{
          "PriorityLevels": ["Critical", "High", "Medium"],
          "DiagnosticApproach": "Systematic problem resolution"
        }},
        "TroubleshootingDetails": {{
          "ErrorSymptoms": ["Detailed problem description"],
          "DiagnosticChecks": ["Step-by-step verification"],
          "RepairProcedures": ["Solution steps"]
        }}
      }}
    }}

    Requirements:
    - Clear error symptom descriptions
    - Systematic diagnostic approach
    - Priority-based solutions
    - 1-2 sentence answers

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }

In [18]:
# Example Usage:
print("--- generate_troubleshooting_qa ---")
test_qa_generators("generate_troubleshooting_qa", content)

--- generate_troubleshooting_qa ---

--- Calling generate_troubleshooting_qa ---
# USER GUIDE ADVANCED TROUBLESHOOTING

This section provides advanced troubleshooting steps for common issues.  Priority levels indicate the urgency of addressing the problem.

**I. Device Power & Charging**

| Error Symptoms                               | Diagnostic Checks                                                                          | Repair Procedures                                                                                                                            | Priority |
|-----------------------------------------------|------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------|---------|
| Device won't turn on                         | Check battery level (Settings > Battery). Check charger and cable. T

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2608.43ms


{
  "Generations": [
    {
      "TroubleshootingQAPairs": []
    }
  ]
}

--- Calling generate_troubleshooting_qa_sonnet ---
{'Generations': {'Questions': [], 'Answers': [], 'RawResponse': '```json\n{\n  "Generations": {\n    "Questions": [\n      "My device won\'t turn on.",\n      "Touchscreen is unresponsive or slow.",\n      "Calls aren\'t connecting.",\n      "People can\'t hear me during calls.",\n      "I\'m experiencing audio echo during calls.",\n      "My cellular data or Wi-Fi keeps disconnecting.",\n      "Battery isn\'t charging properly.",\n      "Battery drains too quickly.",\n      "Camera app is giving errors.",\n      "Picture quality is poor.",\n      "Multimedia files won\'t play.",\n      "Bluetooth is malfunctioning.",\n      "Device won\'t connect to a computer.",\n      "GPS can\'t locate my position.",\n      "Data has been lost from my device.",\n      "There\'s a gap around the device case.",\n      "Device storage is full."\n    ],\n    "Answers": [\n      

### Number 6 generate_feature_explanation_qa

In [19]:
#original
def generate_feature_explanation_qa(text):
    """Technology deep-dive pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Explain Samsung technologies with Q&A pairs covering:
    - Technical specifications - Use cases - Compatibility notes
    Include diagrams in markdown when relevant.
    Text: {text}
    """)
    return model.generate_content(prompt).text


#Deepseek R1
import json
from textwrap import dedent

def generate_feature_explanation_qa_deepseek(text):
    """Generates standardized technical explanations with spec matrices.

    Returns:
        JSON with nested specifications, compatibility tables, and visual references.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create technology deep-dive Q&A pairs with:
    1. Technical specifications in key:value format
    2. Compatibility matrices (Device ↔ Feature Support)
    3. Use case scenarios with required settings
    4. Official diagram references instead of markdown

    Format requirements:
    - JSON with "Generations" containing:
      {{
        "Question": "[Technology inquiry]",
        "Answer": {{
          "Overview": "2-sentence summary",
          "TechnicalSpecs": {{
            "Parameter": "Value",
            "MeasurementUnit": "Hz/nm/mm"
          }},
          "Compatibility": {{
            "GalaxyS24": "Full",
            "GalaxyZFold5": "Partial"
          }},
          "DiagramReference": "KB-12345-fig3"
        }},
        "FeatureType": "Display/Connectivity/AI"
      }}
    - Use standardized units (MHz not Mhz)
    - Replace markdown with Samsung Knowledge Base references

    Example:
    {{
      "Generations": [
        {{
          "Question": "How does Quantum Matrix Technology improve display quality?",
          "Answer": {{
            "Overview": "Uses AI-controlled mini-LED clusters...",
            "TechnicalSpecs": {{
              "DimmingZones": 2048,
              "PeakBrightness": "2500 nits",
              "ResponseTime": "1ms"
            }},
            "Compatibility": {{
              "S95C TV": "Yes",
              "QN900C Neo QLED": "No"
            }},
            "DiagramReference": "KB-DISPLAY-2024-FIG12"
          }},
          "FeatureType": "Display"
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse technical explanation"}


#Google AI STudio
def generate_feature_explanation_qa_google(text):
    """Technology deep-dive pairs in JSON format, with diagram URLs."""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Explain Samsung technologies using Question and Answer pairs from the following support document text, covering:
    - Technical specifications
    - Use cases
    - Compatibility notes

    When diagrams are relevant to illustrate a technology, include references to diagram file names or URLs in the output. Assume diagrams are available as images and you can reference their filenames or URLs.  In the JSON output, include a list of "Diagrams" (URLs or filenames) for each Q&A pair if applicable.

    Use clear and informative language, suitable for users interested in understanding the technology in detail.  Keep answers concise and focused.

    Present the output as a list of technology explanation Q&A pairs in JSON format. Each pair should be a dictionary with keys: "Question", "Answer", "TechnicalSpecifications" (list of specs), "UseCases" (list of use cases), "CompatibilityNotes" (list of notes), and "Diagrams" (list of diagram URLs/filenames).

    Text: {text}

    Output Format (example):
    [
      {{ "Question": "...", "Answer": "...", "TechnicalSpecifications": [...], "UseCases": [...], "CompatibilityNotes": [...], "Diagrams": [...] }},
      {{ "Question": "...", "Answer": "...", "TechnicalSpecifications": [...], "UseCases": [...], "CompatibilityNotes": [...], "Diagrams": [...] }},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        feature_explanation_data = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        feature_explanation_data = [] # Implement robust fallback parsing

    output_json = {
        "Generations": [
            {"FeatureExplanationQAPairs": feature_explanation_data} # Assuming model outputs list of dictionaries
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet
def generate_feature_explanation_qa_sonnet(text):
    """Generate structured technology deep-dive Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create feature explanation Q&A in JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Technical feature inquiry"
        ],
        "Answers": [
          "Comprehensive technology explanation"
        ],
        "Metadata": {{
          "ExplanationDepth": "Technical deep-dive",
          "CoverageAreas": ["Specifications", "Use Cases", "Compatibility"]
        }},
        "TechnicalDetails": {{
          "Specifications": ["Detailed technical parameters"],
          "UseCases": ["Practical application scenarios"],
          "CompatibilityNotes": ["Integration and limitation insights"]
        }},
        "VisualizationSupport": {{
          "DiagramMarkdown": "Optional technical illustration"
        }}
      }}
    }}

    Requirements:
    - In-depth technical explanations
    - Practical use case descriptions
    - Compatibility insights
    - 1-2 sentence answers

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }

In [20]:
# Example Usage:
print("--- generate_feature_explanation_qa ---")
test_qa_generators("generate_feature_explanation_qa", content)

--- generate_feature_explanation_qa ---

--- Calling generate_feature_explanation_qa ---
# Samsung Technologies Explained: Q&A

This document addresses Samsung technologies based on the provided user guide for models SM-S931B, SM-S931B/DS, SM-S936B, SM-S936B/DS, SM-S938B, and SM-S938B/DS (likely Galaxy S25 series).  Due to the broad scope of the user guide, answers are generalized and may not cover every specific detail or nuance.


## Battery Technologies

**Q: What are the charging options for the Galaxy S25 series?**

A: The Galaxy S25 series supports wired charging (via USB Type-C), wireless charging, fast wired charging, fast wireless charging, and wireless power sharing.  It's crucial to use Samsung-approved chargers and cables to avoid damage.

**Q: How does wireless power sharing work?**

A: Wireless power sharing lets you charge other Qi-compatible devices (like Galaxy Buds or Watches) by placing them on the back of your phone.  It's important to align the devices correctly fo

### Number 7 generate_setup_qa

In [21]:
#original
def generate_setup_qa(text):
    """Installation/configuration pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate setup-related Q&A with:
    - Pre-installation checks - Configuration options - Post-setup tests
    Include recommended settings for different user types.
    Text: {text}
    """)
    return model.generate_content(prompt).text


#Deepseek R1
import json
from textwrap import dedent

def generate_setup_qa_deepseek(text):
    """Generates installation guides with user-type presets in JSON format.

    Returns:
        JSON with prerequisites, timed steps, and configuration profiles.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create structured setup Q&A from Samsung docs with:
    1. Prerequisites (tools, OS versions, space requirements)
    2. Configuration profiles: Basic/Advanced/Custom
    3. Step durations and cumulative time estimates
    4. Post-setup verification tests

    Format requirements:
    - JSON with "Generations" containing:
      {{
        "Question": "[Setup scenario]",
        "Answer": {{
          "UserType": "Basic/Advanced",
          "RequiredTools": ["GJ-SETUP45"],
          "SetupDuration": "20-25 mins",
          "ConfigurationSteps": [
            {{
              "Order": 1,
              "Action": "...",
              "Parameters": {{"Resolution": "4K", "FrameRate": 60}}
            }}
          ],
          "PostSetupVerification": ["Check firmware version", "Test HDMI handshake"]
        }},
        "DeviceModel": "QE75QN900C"
      }}
    - Validate model numbers as SM/QE-XXXXX format
    - Sort by setup complexity (Basic first)

    Example:
    {{
      "Generations": [
        {{
          "Question": "How to configure The Frame TV for art mode?",
          "Answer": {{
            "UserType": "Advanced",
            "RequiredTools": ["One Connect Box"],
            "SetupDuration": "15 mins",
            "ConfigurationSteps": [
              {{
                "Order": 1,
                "Action": "Mount Art Store-certified USB",
                "Parameters": {{"Format": "FAT32", "Capacity": "≥64GB"}}
              }}
            ],
            "PostSetupVerification": ["Validate art metadata parsing"]
          }},
          "DeviceModel": "LS03B"
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse setup configuration"}


#Google AI STudio
def generate_setup_qa_google(text):
    """Setup/installation/configuration pairs in JSON format with user-type settings."""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate setup-related Question and Answer pairs from the following Samsung support document text, covering:
    - Pre-installation checks
    - Configuration options
    - Post-setup tests

    For configuration options, include recommended settings tailored for different user types (e.g., "Beginner", "Advanced", "Gamer", etc.) when applicable. If settings are generally applicable, indicate that.

    Use consumer-friendly language. Keep answers concise and step-by-step where relevant.

    Present the output as a list of setup Q&A pairs in JSON format. Each pair should be a dictionary with keys: "Stage" ("Pre-installation Checks", "Configuration Options", "Post-Setup Tests"), "Question", "Answer", and "RecommendedSettings". "RecommendedSettings" should be a dictionary where keys are user types (e.g., "Beginner", "Advanced") and values are the recommended settings for that user type. If no user-type specific settings are relevant, "RecommendedSettings" should be null.

    Text: {text}

    Output Format (example):
    [
      {{ "Stage": "...", "Question": "...", "Answer": "...", "RecommendedSettings": {{ "Beginner": "...", "Advanced": "..." }} }},
      {{ "Stage": "...", "Question": "...", "Answer": "...", "RecommendedSettings": null }},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        setup_data = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        setup_data = [] # Implement robust fallback parsing

    output_json = {
        "Generations": [
            {"SetupQAPairs": setup_data} # Assuming model outputs list of dictionaries
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet
def generate_setup_qa_sonnet(text):
    """Generate structured setup configuration Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create setup Q&A in JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Setup and configuration inquiry"
        ],
        "Answers": [
          "Comprehensive installation guidance"
        ],
        "Metadata": {{
          "SetupStages": ["Pre-installation", "Configuration", "Post-setup"],
          "UserTypeSettings": ["Recommended configurations"]
        }},
        "SetupDetails": {{
          "PreInstallationChecks": ["Essential preparatory steps"],
          "ConfigurationOptions": ["Detailed setup parameters"],
          "PostSetupTests": ["Verification procedures"]
        }}
      }}
    }}

    Requirements:
    - Detailed setup instructions
    - User-type specific guidance
    - Comprehensive verification steps
    - 1-2 sentence answers

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }

In [22]:
# Example Usage:
print("--- generate_setup_qa ---")
test_qa_generators("generate_setup_qa", content)

--- generate_setup_qa ---

--- Calling generate_setup_qa ---
## Samsung Galaxy S25 Series Setup Q&A

This Q&A addresses setup-related questions for the Samsung Galaxy S25, S25+, and S25 Ultra models.  Refer to the user guide for detailed instructions on specific features and functions.


**Pre-Installation Checks:**

**Q: What should I check before I start the initial setup of my Galaxy S25?**

**A:**  Before beginning the initial setup, ensure the following:

1. **Battery Charge:** Your device should have sufficient battery charge (ideally, fully charged).  The initial setup consumes significant power.
2. **SIM Card (Nano-SIM or eSIM):** If using a physical SIM card, have your Nano-SIM card ready and know its tray location (varies slightly between models – see user guide). If using eSIM, obtain the necessary QR code or digital download information from your carrier.
3. **Wi-Fi Network:** Having a Wi-Fi network available will significantly speed up the setup process and allow for downl

### Number 8 generate_error_code_qa

In [23]:
#original
def generate_error_code_qa(text):
    """Error code explanations"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create error code Q&A with:
    - Code interpretations - Immediate actions - Service triggers
    Format codes in bold with severity colors (🔴🟠🟢).
    Text: {text}
    """)
    return model.generate_content(prompt).text

#Deepseek
import json
from textwrap import dedent

def generate_error_code_qa_deepseek(text):
    """Generates machine-actionable error code guides with severity tiers.

    Returns:
        JSON with standardized codes, automated responses, and service flags.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create error code explanations from Samsung docs with:
    1. Code validation using ERR-XXXX format (X=digit)
    2. Severity: Critical(🔴)/High(🟠)/Medium(🟢) mapped to 0-2
    3. Automated system response actions
    4. Service center escalation thresholds

    Format requirements:
    - JSON with "Generations" containing:
      {{
        "ErrorCode": "ERR-0451",
        "Description": "Battery temperature critical",
        "Severity": {{
          "Level": 0,
          "Label": "Critical",
          "HexColor": "#FF0000"
        }},
        "ImmediateResponse": ["Disconnect charger", "Power down"],
        "ServiceRequired": {{
          "Threshold": "Immediate",
          "Code": "SVC-BATT-EMERG"
        }},
        "UserMessage": "Visit Samsung Service within 24hrs"
      }}
    - Validate ErrorCode format: ERR- followed by 4 digits
    - Map colors to standard hex values

    Example:
    {{
      "Generations": [
        {{
          "ErrorCode": "ERR-1123",
          "Description": "Display panel communication failure",
          "Severity": {{
            "Level": 1,
            "Label": "High",
            "HexColor": "#FFA500"
          }},
          "ImmediateResponse": ["Restart device", "Check flex cable"],
          "ServiceRequired": {{
            "Threshold": "If persists after reboot",
            "Code": "SVC-DISP-1123"
          }},
          "UserMessage": "Backup data before service visit"
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse error code protocol"}



#Google AI STudio
def generate_error_code_qa_google(text):
    """Error code explanations in JSON format with severity levels and color codes."""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate Question and Answer pairs explaining Samsung error codes from the following support document text.

    For each error code, include:
    - Code interpretation (what the error means)
    - Immediate actions users can take
    - Service triggers (when to seek professional help)

    Format error codes clearly and indicate their severity level as "Critical", "Warning", or "Informational".  In the JSON output, include a "Severity" field with these levels and optionally a "SeverityColor" field with the corresponding color emoji (🔴 for Critical, 🟠 for Warning, 🟢 for Informational).  In plain text examples within the output, you can format the error codes in bold with the color emojis.

    Use consumer-friendly language. Keep answers concise and action-oriented.

    Present the output as a list of error code Q&A pairs in JSON format. Each pair should be a dictionary with keys: "ErrorCode", "Severity" ("Critical", "Warning", "Informational"), "Interpretation", "ImmediateActions" (list of actions), "ServiceTriggers", and optionally "SeverityColor" (color emoji string).

    Text: {text}

    Output Format (example):
    [
      {{ "ErrorCode": "...", "Severity": "Critical", "Interpretation": "...", "ImmediateActions": [...], "ServiceTriggers": "...", "SeverityColor": "🔴" }},
      {{ "ErrorCode": "...", "Severity": "Warning", "Interpretation": "...", "ImmediateActions": [...], "ServiceTriggers": "...", "SeverityColor": "🟠" }},
      {{ "ErrorCode": "...", "Severity": "Informational", "Interpretation": "...", "ImmediateActions": [...], "ServiceTriggers": "...", "SeverityColor": "🟢" }},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        error_code_data = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        error_code_data = [] # Implement robust fallback parsing

    output_json = {
        "Generations": [
            {"ErrorCodeQAPairs": error_code_data} # Assuming model outputs list of dictionaries
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet
def generate_error_code_qa_sonnet(text):
    """Generate structured error code Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create error code Q&A in JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Specific error code query"
        ],
        "Answers": [
          "Comprehensive error interpretation and resolution"
        ],
        "Metadata": {{
          "ErrorSeverity": ["Critical", "Warning", "Informational"],
          "ResolutionApproach": "Systematic troubleshooting"
        }},
        "ErrorCodeDetails": {{
          "CodeInterpretation": ["Detailed error meaning"],
          "ImmediateActions": ["Recommended quick fixes"],
          "ServiceTriggers": ["Escalation scenarios"]
        }},
        "VisualIndicators": {{
          "SeverityColors": ["🔴 Critical", "🟠 Warning", "🟢 Low Impact"]
        }}
      }}
    }}

    Requirements:
    - Clear error code explanations
    - Actionable resolution steps
    - Severity classification
    - 1-2 sentence answers

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }

In [24]:
# Example Usage:
print("--- generate_error_code_qa ---")
test_qa_generators("generate_error_code_qa", content)

--- generate_error_code_qa ---

--- Calling generate_error_code_qa ---


ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3138.61ms


# USER GUIDE  SM-S931B SM-S931B/DS SM-S936B SM-S936B/DS SM-S938B SM-S938B/DS


## Error Code Q&A

This section provides a simplified Q&A format for common error scenarios.  The user guide provides more detailed troubleshooting steps.

**Note:**  The severity levels are estimations based on the potential impact and are not definitive error codes from the device itself.

**Error Codes & Troubleshooting**

| Error Code | Severity | Code Interpretation | Immediate Actions | Service Triggers |
|---|---|---|---|---|
| **E1:** Battery Not Charging | 🔴 | Device not charging via wired or wireless methods. | Check charger, cable, and port for damage or debris. Try different outlets. Try a different charger/cable. Check battery settings. | If problem persists after trying the immediate actions, contact Samsung Service Centre for battery or charging port issues. |
| **E2:** SIM Card Error | 🟠 | SIM card not detected or malfunctioning. | Check SIM card placement and try restarting the phone. Check 

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2414.01ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2683.96ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2533.06ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2206.77ms


{'Generations': [{'ErrorCode': 'ERR-0001', 'Description': 'Device unresponsive', 'Severity': {'Level': 1, 'Label': 'High', 'HexColor': '#FFA500'}, 'ImmediateResponse': ['Force restart by holding Side and Volume Down buttons for 7 seconds'], 'ServiceRequired': {'Threshold': 'If persists after multiple restarts', 'Code': 'SVC-SYS-0001'}, 'UserMessage': 'Backup data before contacting service.'}, {'ErrorCode': 'ERR-0451', 'Description': 'Battery temperature critical', 'Severity': {'Level': 0, 'Label': 'Critical', 'HexColor': '#FF0000'}, 'ImmediateResponse': ['Disconnect charger', 'Power down'], 'ServiceRequired': {'Threshold': 'Immediate', 'Code': 'SVC-BATT-EMERG'}, 'UserMessage': 'Visit Samsung Service within 24hrs'}, {'ErrorCode': 'ERR-0452', 'Description': 'Battery charging failure', 'Severity': {'Level': 1, 'Label': 'High', 'HexColor': '#FFA500'}, 'ImmediateResponse': ['Try different charger and cable', 'Check charging port for debris'], 'ServiceRequired': {'Threshold': 'If persists af

### Number 9 generate_safety_qa

In [25]:
#original
def generate_safety_qa(text):
    """Safety/compliance pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate safety Q&A covering:
    - Hazard prevention - Compliance standards - Recall information
    Use warning boxes for critical items.
    Text: {text}
    """)
    return model.generate_content(prompt).text


#Deepseek R1
import json
from textwrap import dedent

def generate_safety_qa_deepseek(text):
    """Generates safety alerts with compliance metadata in JSON format.

    Returns:
        JSON with hazard classification, recall batches, and compliance codes.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create safety/compliance Q&A pairs with:
    1. Hazard types: Electrical/Mechanical/Chemical
    2. Compliance standards (ISO/IEC + regional)
    3. Recall data: Batch numbers, date ranges
    4. Severity levels: Critical/High/Medium

    Format requirements:
    - JSON with "Generations" containing:
      {{
        "Question": "[Safety concern]",
        "Answer": {{
          "Description": "...",
          "ComplianceStandards": ["IEC 62368-1"],
          "SeverityLevel": "Critical",
          "RecallInfo": {{
            "AffectedBatches": ["BT2024Q1-45"],
            "DateRange": "2024-03/2024-05"
          }},
          "HazardType": "Electrical"
        }},
        "EmergencyProtocol": ["Unplug immediately", "Evacuate area"]
      }}
    - Validate batch numbers as BTYYYYQX-XX format
    - Sort by SeverityLevel descending

    Example:
    {{
      "Generations": [
        {{
          "Question": "What if my washing machine emits burning smell?",
          "Answer": {{
            "Description": "Potential motor insulation failure...",
            "ComplianceStandards": ["UL 2157"],
            "SeverityLevel": "Critical",
            "RecallInfo": {{
              "AffectedBatches": ["BT2023Q4-12"],
              "DateRange": "2023-11/2024-02"
            }},
            "HazardType": "Electrical"
          }},
          "EmergencyProtocol": ["Cut power", "Contact Samsung Safety Team"]
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse safety protocol"}


#Google AI STudio
def generate_safety_qa_google(text):
    """Safety/compliance pairs in JSON format with 'critical warning' indicator."""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate safety Question and Answer pairs from the following Samsung support document text, covering:
    - Hazard prevention measures
    - Relevant compliance standards and certifications
    - Recall information and how to check for recalls

    For critical safety warnings and hazards, explicitly mark them as such. In the JSON output, use a boolean field "IsCriticalWarning": true/false to indicate if a Q&A pair represents a critical warning. In plain text examples within the output, you can use "WARNING BOX" markers or similar visual cues if helpful.

    Use clear, concise, and direct language appropriate for safety information. Consumer-friendly language is essential.

    Present the output as a list of safety Q&A pairs in JSON format. Each pair should be a dictionary with keys: "IsCriticalWarning" (boolean), "Question", "Answer", and optionally "HazardPrevention" (list of measures), "ComplianceStandards" (list of standards), and "RecallInformation" (string or null).

    Text: {text}

    Output Format (example):
    [
      {{ "IsCriticalWarning": true, "Question": "...", "Answer": "...", "HazardPrevention": [...], "ComplianceStandards": [...], "RecallInformation": "..." }},
      {{ "IsCriticalWarning": false, "Question": "...", "Answer": "...", "HazardPrevention": [...], "ComplianceStandards": [...], "RecallInformation": null }},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        safety_data = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        safety_data = [] # Implement robust fallback parsing

    output_json = {
        "Generations": [
            {"SafetyQAPairs": safety_data} # Assuming model outputs list of dictionaries
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet
def generate_safety_qa_sonnet(text):
    """Generate structured safety and compliance Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create safety Q&A in JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Safety and compliance inquiry"
        ],
        "Answers": [
          "Comprehensive safety guidance"
        ],
        "Metadata": {{
          "SafetyAreas": ["Hazard Prevention", "Compliance", "Recalls"],
          "RiskClassification": "Critical safety information"
        }},
        "SafetyDetails": {{
          "HazardPrevention": ["Critical safety measures"],
          "ComplianceStandards": ["Regulatory requirements"],
          "RecallInformation": ["Potential product safety issues"]
        }},
        "WarningIndicators": {{
          "CriticalWarnings": ["High-risk safety alerts"]
        }}
      }}
    }}

    Requirements:
    - Detailed safety explanations
    - Compliance standard insights
    - Recall information
    - 1-2 sentence answers

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }

In [26]:
# Example Usage:
print("--- generate_safety_qa ---")
test_qa_generators("generate_safety_qa", content)

--- generate_safety_qa ---

--- Calling generate_safety_qa ---
# USER GUIDE

###### SM-S931B SM-S931B/DS SM-S936B SM-S936B/DS SM-S938B SM-S938B/DS

English 01/2025 Rev1 0 www.samsung.com


-----

## Safety Q&A

This section addresses safety precautions, compliance standards, and recall information for your Samsung device.


### Hazard Prevention

**Q: What should I avoid doing to prevent damage to my device's camera?**

**A:**  Avoid exposing the camera lens to strong light sources, such as direct sunlight. This can damage the image sensor, which is irreparable.  Also, when using the speakers, do not place the device close to your ears.


⚠️ **WARNING:**  Direct sunlight or other strong light sources can permanently damage your camera's image sensor.


**Q: How can I prevent connectivity issues and battery drain?**

**A:** Avoid placing metallic stickers on the antenna area, using metallic device covers, or covering the antenna area with your hands while using features like calls or mo

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2023.58ms


{'Generations': [{'Question': 'My phone overheats while fast charging, and a warning message appears.', 'Answer': {'Description': 'The device is overheating due to high power consumption during fast charging. This is a safety precaution to prevent damage to the battery and potential fire hazards.', 'ComplianceStandards': ['IEC 62368-1', 'UL 62368-1'], 'SeverityLevel': 'Critical', 'RecallInfo': {'AffectedBatches': [], 'DateRange': ''}, 'HazardType': 'Electrical'}, 'EmergencyProtocol': ['Disconnect the charger immediately', 'Allow the device to cool down before resuming charging']}, {'Question': 'The screen is unresponsive and the device is frozen.', 'Answer': {'Description': 'The device has experienced a software crash or freeze, potentially caused by a software bug or overloaded memory.  This could lead to data loss if not addressed.', 'ComplianceStandards': ['IEC 62368-1'], 'SeverityLevel': 'High', 'RecallInfo': {'AffectedBatches': [], 'DateRange': ''}, 'HazardType': 'Electrical'}, 'E

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2029.06ms


{
  "Generations": [
    {
      "SafetyQAPairs": []
    }
  ]
}

--- Calling generate_safety_qa_sonnet ---
{'Generations': {'Questions': [], 'Answers': [], 'RawResponse': '```json\n{\n  "SafetyQA": {\n    "Questions": [\n      "What are the safety precautions for using the Samsung Galaxy S25 series?",\n      "What are the compliance standards and regulatory requirements for the device?",\n      "Is there any recall information related to the Samsung Galaxy S25 series?",\n      "What should I do if my device overheats?",\n      "What are the precautions for maintaining water and dust resistance?",\n      "How do I safely charge my device\'s battery?",\n      "What safety measures should I take when using the S Pen (Galaxy S25 Ultra)?"\n    ],\n    "Answers": [\n      "Avoid using the device with a broken glass body; avoid placing the device near your ears when using the speakers; protect the camera lens from strong light sources; do not cover air vents or sensors; and avoid attaching m

### Number 10 generate_compatibility_qa

In [ ]:
#original
def generate_compatibility_qa(text):
    """Device compatibility pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create compatibility Q&A with:
    - Supported models - Version requirements - Regional variations
    Present compatibility as tables when possible.
    Text: {text}
    """)
    return model.generate_content(prompt).text


#Deepseek R1
import json
from textwrap import dedent

def generate_compatibility_qa_deepseek(text):
    """Generates device compatibility matrices with regional versioning in JSON.

    Returns:
        JSON with nested compatibility tables using model codes and ISO regions.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create compatibility Q&A pairs with:
    1. Model numbers in SM/QE-XXXX format
    2. Minimum firmware versions (major.minor.build)
    3. Regional availability using ISO 3166 codes
    4. Feature-specific compatibility flags

    Format requirements:
    - JSON with "Generations" containing:
      {{
        "Question": "[Compatibility inquiry]",
        "Answer": {{
          "CompatibilityMatrix": {{
            "DeviceModel": "SM-S901B",
            "MinimumVersion": "5.1.23",
            "Regions": {{
              "NA": ["QC3.0", "6GHz WiFi"],
              "EU": ["QC3.0", "5G Sub6"]
            }}
          }},
          "Notes": "5G mmWave excluded in EU models"
        }},
        "Category": "Smartphone/Accessory/TV"
      }}
    - Validate model numbers as Samsung's official format
    - Use semantic versioning for firmware

    Example:
    {{
      "Generations": [
        {{
          "Question": "Which Galaxy phones support DeX with 4K monitors?",
          "Answer": {{
            "CompatibilityMatrix": {{
              "DeviceModel": "SM-S928B",
              "MinimumVersion": "6.0.1",
              "Regions": {{
                "Global": ["4K@60Hz via HDMI"],
                "CN": ["4K@30Hz"]
              }}
            }},
            "Notes": "Requires HDMI 2.1 certified cable"
          }},
          "Category": "Smartphone"
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse compatibility matrix"}


#Google AI STudio
def generate_compatibility_qa_google(text):
    """Device compatibility pairs in JSON format, including compatibility tables."""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate Question and Answer pairs about Samsung device compatibility from the following support document text, covering:
    - Supported Samsung models
    - Version requirements (software, OS)
    - Regional variations in compatibility

    Present compatibility information as tables within the answers whenever possible to clearly show model or version compatibility. In the JSON output, represent tables as a list of dictionaries, where each dictionary is a row.

    Use consumer-friendly language. Keep answers concise and present compatibility information in an easy-to-understand format.

    Present the output as a list of compatibility Q&A pairs in JSON format. Each pair should be a dictionary with keys: "Question", "Answer", "CompatibilityTable" (list of row dictionaries - see example format), "VersionRequirements" (string or null), and "RegionalVariations" (string or null). If no table is applicable, "CompatibilityTable" should be null.

    Text: {text}

    Output Format (example, tables as list of dictionaries):
    [
      {{
        "Question": "...",
        "Answer": "...",
        "CompatibilityTable": [
          {{ "Column 1 Header": "Value 1", "Column 2 Header": "Value 2" }},
          {{ "Column 1 Header": "Value 3", "Column 2 Header": "Value 4" }},
          ...
        ],
        "VersionRequirements": "...",
        "RegionalVariations": "..."
      }},
      {{
        "Question": "...",
        "Answer": "...",
        "CompatibilityTable": null,
        "VersionRequirements": "...",
        "RegionalVariations": "..."
      }},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        compatibility_data = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        compatibility_data = [] # Implement robust fallback parsing

    output_json = {
        "Generations": [
            {"CompatibilityQAPairs": compatibility_data} # Assuming model outputs list of dictionaries
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet
def generate_compatibility_qa_sonnet(text):
    """Generate structured device compatibility Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create compatibility Q&A in JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Device compatibility inquiry"
        ],
        "Answers": [
          "Comprehensive compatibility explanation"
        ],
        "Metadata": {{
          "CompatibilityTypes": ["Model Support", "Version Requirements", "Regional Variations"]
        }},
        "CompatibilityDetails": {{
          "SupportedModels": ["Detailed model list"],
          "VersionRequirements": ["Minimum/optimal version specifications"],
          "RegionalVariations": ["Geographic compatibility notes"]
        }},
        "CompatibilityTable": {{
          "Format": "Tabular representation of compatibility data"
        }}
      }}
    }}

    Requirements:
    - Comprehensive compatibility information
    - Model and version specifics
    - Regional considerations
    - 1-2 sentence answers

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }

In [ ]:
# Example Usage:
print("--- generate_compatibility_qa ---")
test_qa_generators("generate_compatibility_qa", content)

### Number 11 generate_maintenance_qa

In [ ]:
#original
def generate_maintenance_qa(text):
    """Preventive care pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate maintenance Q&A including:
    - Service intervals - Cleaning procedures - Part replacements
    Add maintenance checklists with timelines.
    Text: {text}
    """)
    return model.generate_content(prompt).text


#Deepseek R1
import json
from textwrap import dedent

def generate_maintenance_qa_deepseek(text):
    """Generates maintenance schedules with part/tool validation in JSON format.

    Returns:
        JSON with nested maintenance tiers, validated part SKUs, and interval units.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create maintenance Q&A pairs with:
    1. Maintenance types: Preventive/Predictive/Corrective
    2. Interval formats: Days/UsageHours/Cycles
    3. Samsung part numbers (SPX-XXXX format)
    4. Tool requirements with official SKUs

    Format requirements:
    - JSON with "Generations" containing:
      {{
        "Question": "[Maintenance task]",
        "Answer": {{
          "MaintenanceType": "Preventive",
          "Interval": {{
            "Value": 180,
            "Unit": "Days",
            "Condition": "NormalUsage"
          }},
          "Procedures": [
            {{
              "Step": 1,
              "Action": "Clean air vents",
              "Tools": ["GJ-CLN45"],
              "PartSKU": "SPX-0987"
            }}
          ],
          "Criticality": "High"
        }},
        "LastServicedDate": "2024-03-15"
      }}
    - Validate part numbers as SPX-XXXX
    - Sort by Criticality then Interval.Value

    Example:
    {{
      "Generations": [
        {{
          "Question": "How often should I service my Bespoke refrigerator?",
          "Answer": {{
            "MaintenanceType": "Predictive",
            "Interval": {{
              "Value": 365,
              "Unit": "Days",
              "Condition": "HeavyIceUsage"
            }},
            "Procedures": [
              {{
                "Step": 1,
                "Action": "Defrost ice maker",
                "Tools": ["GJ-DEF12"],
                "PartSKU": "SPX-4567"
              }}
            ],
            "Criticality": "Medium"
          }},
          "LastServicedDate": "2023-09-01"
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse maintenance protocol"}


#Google AI STudio
def generate_maintenance_qa_google(text):
    """Preventive maintenance pairs in JSON format with checklists and timelines."""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate maintenance Question and Answer pairs from the following Samsung support document text, including:
    - Service intervals (how often maintenance is needed)
    - Cleaning procedures (how to clean specific parts or the device)
    - Part replacements (when and which parts need replacement)

    For key maintenance tasks, create maintenance checklists with recommended timelines for each task.  Present timelines clearly (e.g., "Monthly", "Every 6 Months", "Annually").

    Use consumer-friendly language. Keep answers concise and provide actionable maintenance advice.

    Present the output as a list of maintenance Q&A pairs in JSON format. Each pair should be a dictionary with keys: "Question", "Answer", "ServiceInterval" (string or null), "CleaningProcedure" (string or null), "PartReplacement" (string or null), and "MaintenanceChecklist" (dictionary - see example format below, or null if no checklist is applicable).  "MaintenanceChecklist" should contain a "Timeline" (overall checklist timeline) and "Items" (list of checklist item dictionaries, each with "Task", "Timeline", and "Details").

    Text: {text}

    Output Format (example, checklists as nested dictionaries):
    [
      {{
        "Question": "...",
        "Answer": "...",
        "ServiceInterval": "...",
        "CleaningProcedure": "...",
        "PartReplacement": "...",
        "MaintenanceChecklist": {{
          "Timeline": "...",
          "Items": [
            {{ "Task": "...", "Timeline": "...", "Details": "..." }},
            {{ "Task": "...", "Timeline": "...", "Details": "..." }},
            ...
          ]
        }}
      }},
      {{
        "Question": "...",
        "Answer": "...",
        "ServiceInterval": null,
        "CleaningProcedure": null,
        "PartReplacement": null,
        "MaintenanceChecklist": null
      }},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        maintenance_data = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        maintenance_data = [] # Implement robust fallback parsing

    output_json = {
        "Generations": [
            {"MaintenanceQAPairs": maintenance_data} # Assuming model outputs list of dictionaries
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet
def generate_maintenance_qa_sonnet(text):
    """Generate structured preventive maintenance Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create maintenance Q&A in JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Maintenance and care inquiry"
        ],
        "Answers": [
          "Comprehensive maintenance guidance"
        ],
        "Metadata": {{
          "MaintenanceTypes": ["Service Intervals", "Cleaning", "Part Replacement"]
        }},
        "MaintenanceDetails": {{
          "ServiceIntervals": ["Recommended maintenance schedule"],
          "CleaningProcedures": ["Detailed cleaning instructions"],
          "PartReplacements": ["Component replacement guidelines"]
        }},
        "MaintenanceChecklist": {{
          "Timeline": "Periodic maintenance schedule"
        }}
      }}
    }}

    Requirements:
    - Detailed maintenance guidance
    - Service interval specifics
    - Cleaning and replacement procedures
    - 1-2 sentence answers

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }

In [ ]:
# Example Usage:
print("--- generate_maintenance_qa ---")
test_qa_generators("generate_maintenance_qa", content)

### Number 12 generate_warranty_qa

In [ ]:
#original
def generate_warranty_qa(text):
    """Warranty/service pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create warranty Q&A covering:
    - Coverage details - Claim processes - Void conditions
    Include regional policy variations.
    Text: {text}
    """)
    return model.generate_content(prompt).text


#Deepseek R1
import json
from textwrap import dedent

def generate_warranty_qa_deepseek(text):
    """Generates warranty rules with policy validation in JSON format.

    Returns:
        JSON with coverage timelines, ISO regions, and machine-readable void terms.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create warranty Q&A pairs with:
    1. Policy numbers in WR-YYYY-REG format (e.g., WR-2024-US)
    2. Coverage periods as start/end dates
    3. Void conditions as codified rules (MODIFIED_HARDWARE = void)
    4. Regional terms using ISO 3166 codes

    Format requirements:
    - JSON with "Generations" containing:
      {{
        "Question": "[Warranty inquiry]",
        "Answer": {{
          "PolicyNumber": "WR-2024-EU",
          "CoveragePeriod": {{
            "Start": "2024-01-01",
            "End": "2026-12-31",
            "DurationDays": 1095
          }},
          "Regions": ["DE", "FR", "IT"],
          "VoidConditions": [
            {{
              "Code": "LIQUID_DAMAGE",
              "Description": "Water ingress signs present"
            }}
          ],
          "ClaimProcess": ["Submit form W-45", "Provide proof of purchase"]
        }},
        "ProductCategory": "Smartphone/HomeAppliance"
      }}
    - Validate PolicyNumber format
    - Sort by CoveragePeriod.DurationDays descending

    Example:
    {{
      "Generations": [
        {{
          "Question": "Does installing third-party parts void my TV warranty?",
          "Answer": {{
            "PolicyNumber": "WR-2024-GLOBAL",
            "CoveragePeriod": {{
              "Start": "2024-03-01",
              "End": "2027-03-01",
              "DurationDays": 1096
            }},
            "Regions": ["GLOBAL"],
            "VoidConditions": [
              {{
                "Code": "UNAUTH_PARTS",
                "Description": "Non-Samsung display components detected"
              }}
            ],
            "ClaimProcess": ["Contact ASC", "Run diagnostic tool"]
          }},
          "ProductCategory": "TV"
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse warranty policy"}


#Google AI STudio
def generate_warranty_qa_google(text):
    """Warranty/service pairs in JSON format, including regional policy variations."""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate warranty Question and Answer pairs from the following Samsung support document text, covering:
    - Warranty coverage details (what is covered, duration)
    - Warranty claim processes (how to initiate a claim)
    - Conditions that void the warranty

    Crucially, include regional policy variations. Warranty policies often differ significantly by region.  Represent these regional differences clearly in the output. In the JSON output, structure regional variations under a "RegionalPolicyVariations" key, with regions (e.g., "USA", "Europe", "Asia") as keys and policy details as values.

    Use consumer-friendly language. Keep answers concise and provide accurate warranty information.

    Present the output as a list of warranty Q&A pairs in JSON format. Each pair should be a dictionary with keys: "Question", "Answer", "CoverageDetails" (list of strings or null), "ClaimProcess" (string or null), "VoidConditions" (list of strings or null), and "RegionalPolicyVariations" (dictionary - see example format below, or null if no regional variations are mentioned in the Q&A pair).

    Text: {text}

    Output Format (example, regional variations as nested dictionary):
    [
      {{
        "Question": "...",
        "Answer": "...",
        "CoverageDetails": [...],
        "ClaimProcess": "...",
        "VoidConditions": [...],
        "RegionalPolicyVariations": {{
          "Region 1 Name": {{ "Policy Detail 1": "Value 1", "Policy Detail 2": "Value 2" }},
          "Region 2 Name": {{ "Policy Detail 1": "Value 3", "Policy Detail 2": "Value 4" }},
          ...
        }}
      }},
      {{
        "Question": "...",
        "Answer": "...",
        "CoverageDetails": null,
        "ClaimProcess": null,
        "VoidConditions": null,
        "RegionalPolicyVariations": null
      }},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        warranty_data = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        warranty_data = [] # Implement robust fallback parsing

    output_json = {
        "Generations": [
            {"WarrantyQAPairs": warranty_data} # Assuming model outputs list of dictionaries
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet
def generate_warranty_qa_sonnet(text):
    """Generate structured warranty and service Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create warranty Q&A in JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Warranty and service inquiry"
        ],
        "Answers": [
          "Comprehensive warranty guidance"
        ],
        "Metadata": {{
          "WarrantyAreas": ["Coverage", "Claims", "Void Conditions"]
        }},
        "WarrantyDetails": {{
          "CoverageSpecifics": ["Detailed warranty terms"],
          "ClaimProcess": ["Step-by-step claim procedures"],
          "VoidConditions": ["Warranty invalidation scenarios"]
        }},
        "RegionalVariations": {{
          "PolicyDifferences": "Geographic policy variations"
        }}
      }}
    }}

    Requirements:
    - Detailed warranty information
    - Claim process insights
    - Regional policy differences
    - 1-2 sentence answers

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }

In [ ]:
# Example Usage:
print("--- generate_warranty_qa ---")
test_qa_generators("generate_warranty_qa", content)

### Number 13 generate_update_qa

In [ ]:
#original
def generate_update_qa(text):
    """Software update pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create update-related Q&A including:
    - Version histories - Rollback methods - Security patches
    Add update compatibility matrices.
    Text: {text}
    """)
    return model.generate_content(prompt).text


#Deepseek R1
import json
from textwrap import dedent

def generate_update_qa_deepseek(text):
    """Generates update guides with version validation and security metadata.

    Returns:
        JSON with semantic versioning, CVE tracking, and rollback metrics.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create software update Q&A pairs with:
    1. Semantic versioning (major.minor.patch)
    2. CVE numbers for security patches
    3. Rollback success rates (%) and prerequisites
    4. Model-specific compatibility using SM-XXXX codes

    Format requirements:
    - JSON with "Generations" containing:
      {{
        "Question": "[Update inquiry]",
        "Answer": {{
          "CurrentVersion": "6.1.1",
          "TargetVersion": "7.0.0",
          "SecurityPatches": [
            {{
              "CVE": "CVE-2024-1234",
              "Severity": "Critical",
              "PatchDate": "2024-03-15"
            }}
          ],
          "Compatibility": {{
            "DeviceModel": "SM-S901B",
            "MinOSVersion": "5.0.0"
          }},
          "Rollback": {{
            "Possible": true,
            "SuccessRate": 92,
            "Steps": ["Enter recovery", "Select rollback image"]
          }},
          "UpdateSize": "1.4GB"
        }},
        "ReleaseChannel": "Stable/Beta"
      }}
    - Validate versions as X.Y.Z numbers
    - Sort by TargetVersion descending

    Example:
    {{
      "Generations": [
        {{
          "Question": "How to revert from One UI 6.1 to 6.0?",
          "Answer": {{
            "CurrentVersion": "6.1.0",
            "TargetVersion": "6.0.1",
            "SecurityPatches": [],
            "Compatibility": {{
              "DeviceModel": "SM-S918B",
              "MinOSVersion": "5.1.2"
            }},
            "Rollback": {{
              "Possible": true,
              "SuccessRate": 85,
              "Steps": ["Backup data", "Download rollback package"]
            }},
            "UpdateSize": "2.1GB"
          }},
          "ReleaseChannel": "Stable"
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse update protocol"}


#Google AI STudio
def generate_update_qa_google(text):
    """Software update pairs in JSON format, including compatibility matrices."""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate Question and Answer pairs about Samsung software updates from the following support document text, including:
    - Version histories and key changes in each version
    - Methods to rollback or revert to previous versions
    - Security patches included in updates

    Present update compatibility information as compatibility matrices whenever possible to clearly show which devices are compatible with which updates. In the JSON output, represent matrices as a list of dictionaries, where each dictionary is a row.

    Use consumer-friendly language. Keep answers concise and provide accurate update information.

    Present the output as a list of update Q&A pairs in JSON format. Each pair should be a dictionary with keys: "Question", "Answer", "VersionHistories" (list of strings or null), "RollbackMethods" (list of strings or null), "SecurityPatches" (list of strings or null), and "UpdateCompatibilityMatrix" (list of row dictionaries - see example format below, or null if no matrix is applicable).

    Text: {text}

    Output Format (example, matrices as list of dictionaries):
    [
      {{
        "Question": "...",
        "Answer": "...",
        "VersionHistories": [...],
        "RollbackMethods": [...],
        "SecurityPatches": [...],
        "UpdateCompatibilityMatrix": [
          {{ "Column 1 Header": "Value 1", "Column 2 Header": "Value 2" }},
          {{ "Column 1 Header": "Value 3", "Column 2 Header": "Value 4" }},
          ...
        ]
      }},
      {{
        "Question": "...",
        "Answer": "...",
        "VersionHistories": null,
        "RollbackMethods": null,
        "SecurityPatches": null,
        "UpdateCompatibilityMatrix": null
      }},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        update_data = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        update_data = [] # Implement robust fallback parsing

    output_json = {
        "Generations": [
            {"UpdateQAPairs": update_data} # Assuming model outputs list of dictionaries
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet
def generate_update_qa_sonnet(text):
    """Generate structured software update Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create update Q&A in JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Software update inquiry"
        ],
        "Answers": [
          "Comprehensive update guidance"
        ],
        "Metadata": {{
          "UpdateTypes": ["Version History", "Rollback", "Security Patches"]
        }},
        "UpdateDetails": {{
          "VersionHistory": ["Detailed update progression"],
          "RollbackMethods": ["Reversal procedures"],
          "SecurityPatches": ["Critical security updates"]
        }},
        "CompatibilityMatrix": {{
          "DeviceModelSupport": "Update compatibility information"
        }}
      }}
    }}

    Requirements:
    - Update version details
    - Rollback process
    - Security update insights
    - 1-2 sentence answers

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }

In [ ]:
# Example Usage:
print("--- generate_update_qa ---")
test_qa_generators("generate_update_qa", content)

### Number 14 generate_preventive_qa

In [ ]:
#original
def generate_preventive_qa(text):
    """Failure prevention pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate preventive care Q&A with:
    - Best practices - Early warning signs - Monitoring tips
    Format as do's/don'ts lists.
    Text: {text}
    """)
    return model.generate_content(prompt).text


#Deepseek R1
import json
from textwrap import dedent

def generate_preventive_qa_deepseek(text):
    """Generates preventive Q&A with risk prioritization in JSON format.

    Returns:
        JSON with nested maintenance categories, severity levels, and action lists.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate failure prevention Q&A from this text with:
    1. Do’s/don’ts for device longevity
    2. Early hardware/software warning signs
    3. Monitoring frequency guidelines

    Format requirements:
    - Return JSON with "Generations" containing:
      {{
        "Question": "...",
        "Answer": {{
          "Do": ["list", "items"],
          "Dont": ["list", "items"]
        }},
        "Category": "Battery/Screen/Software",
        "SeverityLevel": "High/Medium/Low",
        "MonitoringFrequency": "Daily/Weekly/Monthly"
      }}
    - Prioritize entries by SeverityLevel (High first)

    Example:
    {{
      "Generations": [
        {{
          "Question": "How to prevent battery swelling in Galaxy phones?",
          "Answer": {{
            "Do": ["Use 25W official charger", "Store at 15°-25°C"],
            "Dont": ["Charge overnight", "Use while gaming >1hr"]
          }},
          "Category": "Battery",
          "SeverityLevel": "High",
          "MonitoringFrequency": "Weekly"
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse preventive maintenance data"}


#Google AI STudio
def generate_preventive_qa_google(text):
    """Failure prevention pairs in JSON format, accommodating do's/don'ts lists"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate preventive care information from the following Samsung support document text.
    Focus on:
    - Best practices for device maintenance and longevity
    - Early warning signs of potential issues
    - Monitoring tips to proactively identify problems

    Format preventive advice primarily as do's and don'ts lists where suitable for clear, actionable guidance. For other information, use Question and Answer pairs.
    Use consumer-friendly language and keep answers concise (1-2 sentences for Q&A, short phrases for do's/don'ts).

    Present the output in JSON format. Structure it to include topics, and for each topic, indicate if it's presented as "DosAndDonts" or "Q&A", and provide the corresponding lists or Q&A pairs.

    Text: {text}

    Output Format (example - model should generate similar structure):
    [
      {{
        "Topic": "Topic Name",
        "Format": "DosAndDonts",
        "Dos": ["Do this...", "Do that..."],
        "Donts": ["Don't do this...", "Don't do that..."]
      }},
      {{
        "Topic": "Another Topic",
        "Format": "Q&A",
        "Questions": ["Question 1...", "Question 2..."],
        "Answers": ["Answer 1...", "Answer 2..."]
      }}
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        preventive_care_data = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        preventive_care_data = [] # Implement a robust fallback text parsing if needed

    output_json = {
        "Generations": [
            {"PreventiveCareQAPairs": preventive_care_data} # Assuming the model directly outputs the list as in "Output Format"
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet
def generate_preventive_qa_sonnet(text):
    """Generate structured preventive care Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate preventive care Q&A in JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Best practice identification question",
          "Early warning signs inquiry",
          "Monitoring technique question"
        ],
        "Answers": [
          "Comprehensive best practices explanation",
          "Detailed early warning signs description",
          "Effective monitoring techniques guidance"
        ],
        "Metadata": {{
          "PreventionCategories": ["Best Practices", "Warning Signs", "Monitoring"],
          "ApproachType": "Proactive Care"
        }},
        "DosDonts": {{
          "Dos": ["Recommended actions", "Preventive measures"],
          "Donts": ["Actions to avoid", "Potential risk behaviors"]
        }}
      }}
    }}

    Key Requirements:
    - Provide clear, actionable preventive guidance
    - Include specific, measurable recommendations
    - 1-2 sentence answers
    - Balance technical accuracy with user comprehension

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }



In [ ]:
# Example Usage:
print("--- generate_preventive_qa ---")
test_qa_generators("generate_preventive_qa", content)



### Number 15 generate_accessory_qa

In [ ]:
#original
def generate_structured_accessory_qa(text):
    """Accessory integration pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Create accessory Q&A covering:
    - Pairing steps - Compatibility checks - Optimal usage
    Include specific model numbers and SKUs.
    Text: {text}
    """)
    return model.generate_content(prompt).text


#Deepseek R1
import json
from textwrap import dedent

def generate_structured_accessory_qa_deepseek(text):
    """Generates accessory Q&A with technical specs in standardized format.

    Returns:
        JSON with nested device models, SKUs, and setup complexity levels.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate accessory integration Q&A from this text with:
    1. Pairing instructions (Bluetooth/USB-C/Wireless Dock)
    2. Compatibility matrices (Galaxy model-to-accessory matching)
    3. Usage best practices (charging limits, cleaning methods)

    Format requirements:
    - Use JSON with "Generations" containing list of:
      {{
        "Question": "...",
        "Answer": "...",
        "DeviceModel": "Galaxy S24 Ultra",
        "SKU": "EJ-TG980BBEGUS",
        "ConnectionType": "Wireless Charging",
        "Complexity": "Medium"
      }}
    - Order by setup complexity (Simple -> Complex)
    - Validate SKUs match Samsung's 10-char alphanumeric format

    Example:
    {{
      "Generations": [
        {{
          "Question": "How to pair Galaxy Buds2 Pro with S23 FE?",
          "Answer": "Open case > tap phone notification > follow in-app steps",
          "DeviceModel": "SM-S711B",
          "SKU": "EI-R510NBEGUS",
          "ConnectionType": "Bluetooth 5.3",
          "Complexity": "Simple"
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse accessory integration data"}


#Google AI STudio
def generate_structured_accessory_qa_google(text):
    """Accessory integration pairs in JSON format"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate Question and Answer pairs about Samsung accessory integration from the following support document text.
    Focus on:
    - Pairing steps for accessories
    - Compatibility checks between devices and accessories
    - Optimal usage tips and features for accessories

    Whenever relevant, include specific Samsung model numbers and/or SKUs for devices and accessories mentioned.
    Format each question and answer pair clearly. Use consumer-friendly language and keep answers concise (1-2 sentences).

    Present the output as a list of dictionaries in JSON format. Each dictionary should have the keys "Question", "Answer", "Model/SKU". If no specific model or SKU is relevant to a Q&A pair, the "Model/SKU" key can be null or omitted.

    Text: {text}

    Output Format:
    [
      {{"Question": "...", "Answer": "...", "Model/SKU": "ModelXYZ"}},
      {{"Question": "...", "Answer": "...", "Model/SKU": null}},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        qa_list = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        qa_list = [] # Implement a robust fallback text parsing as needed

    output_json = {
        "Generations": [
            {
                "Questions": [qa_pair.get("Question") for qa_pair in qa_list],
                "Answers": [qa_pair.get("Answer") for qa_pair in qa_list],
                "Model_SKUs": [qa_pair.get("Model/SKU", None) for qa_pair in qa_list] # Changed key to "Model_SKUs" for clarity in list context
            }
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet
def generate_structured_accessory_qa_sonnet(text):
    """Generate structured accessory integration Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate accessory Q&A in precise JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Pairing procedure question",
          "Compatibility verification question",
          "Optimal usage inquiry"
        ],
        "Answers": [
          "Detailed pairing steps with specific model guidance",
          "Comprehensive compatibility explanation",
          "Optimal usage recommendations"
        ],
        "Metadata": {{
          "CoverageAreas": ["Pairing", "Compatibility", "Usage"],
          "DetailLevel": "Technical and user-friendly"
        }}
      }}
    }}

    Requirements:
    - Include specific model numbers
    - Provide clear, step-by-step guidance
    - 1-2 sentence answers
    - Technical accuracy with user comprehension

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }


In [ ]:
# Example Usage:
print("--- generate_structured_accessory_qa_sonnet ---")
test_qa_generators("generate_structured_accessory_qa", content)

### Number 16 generate_legal_qa

In [ ]:
#original
def generate_structured_legal_qa(text):
    """Regulatory/compliance pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate legal Q&A with:
    - Certification details - Disposal guidelines - Privacy compliance
    Include regulation numbers and citations.
    Text: {text}
    """)


#Deepseek R1
import json
from textwrap import dedent

def generate_structured_legal_qa_deepseek(text):
    """Generates compliance Q&A with standardized legal references.

    Returns:
        JSON with regulation IDs, full citations, and nested Q&A pairs.
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate legal/compliance Q&A pairs from this text with:
    1. Certification standards (e.g., ISO/IEC numbers)
    2. E-waste disposal regulations (WEEE/RoHS directives)
    3. Privacy frameworks (GDPR, CCPA articles)

    Format requirements:
    - Use strict JSON format with "Generations" as root key
    - Each entry must contain:
      {{
        "Question": "...",
        "Answer": "...",
        "RegulationID": "GDPR Article 12",
        "FullCitation": "Regulation (EU) 2016/679"
      }}
    - Sort by regulation importance (EU directives first)

    Example:
    {{
      "Generations": [
        {{
          "Question": "How should I dispose of Samsung batteries?",
          "Answer": "Follow WEEE Directive 2012/19/EU through certified recyclers.",
          "RegulationID": "WEEE Directive",
          "FullCitation": "Directive 2012/19/EU on waste electrical equipment"
        }}
      ]
    }}

    Text: {text}
    """)

    try:
        response = model.generate_content(prompt).text
        response = response.strip("` \n").replace("json", "")
        return json.loads(response)
    except json.JSONDecodeError:
        return {"error": "Failed to parse legal compliance data"}


#Google AI STudio
def generate_structured_legal_qa_google(text):
    """Regulatory/compliance pairs in JSON format"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate legal Question and Answer pairs from the following Samsung support document text.
    Focus on:
    - Certification details (e.g., FCC, CE, UL)
    - Disposal guidelines (e.g., WEEE, battery disposal)
    - Privacy compliance (e.g., GDPR, CCPA)

    Include relevant regulation numbers and/or citations where applicable.
    Format each question and answer pair clearly. Aim for consumer-friendly language and keep answers concise (1-2 sentences).

    Present the output as a list of dictionaries in JSON format. Each dictionary should have the keys "Question" and "Answer". If applicable, include a "Citation" key with the regulation number or citation. If no citation is relevant, the "Citation" key can be null or omitted.

    Text: {text}

    Output Format:
    [
      {{"Question": "...", "Answer": "...", "Citation": "..."}},
      {{"Question": "...", "Answer": "...", "Citation": null}},
      ...
    ]
    """)
    response_text = model.generate_content(prompt).text

    try:
        qa_list = json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: JSON parse failed, attempting fallback text extraction.")
        qa_list = [] # Implement a robust fallback text parsing as needed, similar to before.

    output_json = {
        "Generations": [
            {
                "Questions": [qa_pair.get("Question") for qa_pair in qa_list], # Use .get to handle potentially missing keys
                "Answers": [qa_pair.get("Answer") for qa_pair in qa_list],
                "Citations": [qa_pair.get("Citation", None) for qa_pair in qa_list] # Get citation, default None if missing
            }
        ]
    }
    return json.dumps(output_json, indent=2)


#Sonnet

def generate_structured_legal_qa_sonnet(text):
    """Generate structured legal compliance Q&A pairs"""
    model = genai.GenerativeModel('gemini-1.5-flash')
    prompt = dedent(f"""
    Generate legal Q&A pairs in strict JSON format:
    {{
      "Generations": {{
        "Questions": [
          "Legal question addressing certification",
          "Regulatory compliance question"
        ],
        "Answers": [
          "Detailed answer with specific regulation citations",
          "Precise compliance explanation"
        ],
        "Metadata": {{
          "ComplianceAreas": ["Certification", "Disposal", "Privacy"],
          "RegulationScope": "Comprehensive legal overview"
        }}
      }}
    }}

    Key Requirements:
    - Include precise regulation numbers
    - Cite specific legal standards
    - 1-2 sentence answers
    - Technical but understandable language

    Text: {text}
    """)

    response = model.generate_content(prompt)

    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        return {
            "Generations": {
                "Questions": [],
                "Answers": [],
                "RawResponse": response.text
            }
        }


In [ ]:
# Example Usage:
print("--- generate_structured_legal_qa_sonnet ---")
test_qa_generators("generate_structured_legal_qa", content)